In [7]:
import math
import argparse
import sqlite3
import rosbag2_py
import os
import string
from rosidl_runtime_py.utilities import get_message
from rclpy.serialization import deserialize_message, serialize_message
from operator import itemgetter
from os import path
from tqdm import tqdm
from calendar import c
import pygeodesy
import math
from std_msgs.msg import Header
from nav_msgs.msg import Odometry
from novatel_oem7_msgs.msg import BESTPOS, BESTVEL, Oem7Header
from scipy.spatial.transform import Rotation as R

In [8]:
class BagFileParser():
    def __init__(self, bag_file):
        self.conn = sqlite3.connect(bag_file)
        self.cursor = self.conn.cursor()

        ## create a message type map
        topics_data = self.cursor.execute("SELECT id, name, type FROM topics").fetchall()
        self.topic_type = {name_of:type_of for id_of,name_of,type_of in topics_data}
        self.topic_id = {name_of:id_of for id_of,name_of,type_of in topics_data}
        # Ignore topics with undefined message structures
        self.topic_msg_message = {}
        for id_of,name_of,type_of in topics_data:
            try:
                self.topic_msg_message[name_of] = get_message(type_of)
            except:
                self.topic_msg_message[name_of] = None

    def __del__(self):
        self.conn.close()

    # Return [(timestamp0, message0), (timestamp1, message1), ...]
    def get_messages(self, topic_name):

        topic_id = self.topic_id[topic_name]
        # Get from the db
        rows = self.cursor.execute("SELECT timestamp, data FROM messages WHERE topic_id = {}".format(topic_id)).fetchall()
        # Deserialise all and timestamp them
        return [ (timestamp,deserialize_message(data, self.topic_msg_message[topic_name])) for timestamp,data in rows]

    # def write_messages(self, topic_name, data):

        

In [9]:
def euler_from_quaternion(x, y, z, w):
        """
        Convert a quaternion into euler angles (roll, pitch, yaw)
        roll is rotation around x in radians (counterclockwise)
        pitch is rotation around y in radians (counterclockwise)
        yaw is rotation around z in radians (counterclockwise)
        """
        t0 = +2.0 * (w * x + y * z)
        t1 = +1.0 - 2.0 * (x * x + y * y)
        roll_x = math.atan2(t0, t1)
     
        t2 = +2.0 * (w * y - z * x)
        t2 = +1.0 if t2 > +1.0 else t2
        t2 = -1.0 if t2 < -1.0 else t2
        pitch_y = math.asin(t2)
     
        t3 = +2.0 * (w * z + x * y)
        t4 = +1.0 - 2.0 * (y * y + z * z)
        yaw_z = math.atan2(t3, t4)
     
        return roll_x, pitch_y, yaw_z # in radians

In [10]:
def gen_local_odom(gps_messages, vel_messages):

    deg2rad = math.pi/180.0

    local_odom_arr = []
    LVMS_Origin = (36.272371177449344, -115.01030828834901, 594)
    gpsmap = pygeodesy.LocalCartesian(LVMS_Origin[0], LVMS_Origin[1], LVMS_Origin[2])

    prev_pos = [0.0,0.0,0.0]

    for ros_time, bestpos in gps_messages:

        local_odom = Odometry()
        bestvel = BESTVEL()

        ros_time_vel = 0

        # Parse BESTPOS

        ros_header = bestpos.header
        ros_frame  = ros_header.frame_id
        gps_header = bestpos.nov_header
        gps_time   = gps_header.gps_week_milliseconds
        lat        = bestpos.lat
        lon        = bestpos.lon
        hgt        = bestpos.hgt
        lat_stdev  = bestpos.lat_stdev  
        lon_stdev  = bestpos.lon_stdev  
        hgt_stdev  = bestpos.hgt_stdev

        novel = True
        # Find Corresponding BESTVEL
        for time,msg in vel_messages:
            if msg.nov_header.gps_week_milliseconds == gps_time:
                ros_time_vel = time
                bestvel = msg
                novel = False
                break
            if msg.nov_header.gps_week_milliseconds > gps_time:
                break 

        # Parse BESTVEL
        ros_header_v = bestvel.header
        gps_header_v = bestvel.nov_header
        gps_time_v   = gps_header_v.gps_week_milliseconds
        trk_gnd      = bestvel.trk_gnd
        latency      = bestvel.latency
        hor_speed    = bestvel.hor_speed
        ver_speed    = bestvel.ver_speed

        # GPS 2 LOCAL CARTESIAN
        local_tuple = gpsmap.forward(lat, lon, hgt)
        if novel:
            heading = math.atan2(local_tuple[1] - prev_pos[1], local_tuple[0] - prev_pos[0]) - math.pi/2
            # quat    = R.from_euler('z',heading).as_quat()
        else:
            heading = -trk_gnd*deg2rad
        
        trk_r  = R.from_euler('z',heading)
        base_r = R.from_euler('z', math.pi/2)
        quat = (trk_r*base_r).as_quat()
        
        
        rot_stdev = (math.pi/2*180)

        

        prev_pos[0] = local_tuple[0]
        prev_pos[1] = local_tuple[1]
        prev_pos[2] = local_tuple[2]


        # Populate Odom Message

        local_odom.header                  = ros_header
        local_odom.header.frame_id         = 'map'
        local_odom.child_frame_id          = 'gps'
        local_odom.pose.pose.position.x    = local_tuple[0]
        local_odom.pose.pose.position.y    = local_tuple[1]
        local_odom.pose.pose.position.z    = local_tuple[2]
        local_odom.pose.pose.orientation.x = quat[0]
        local_odom.pose.pose.orientation.y = quat[1]
        local_odom.pose.pose.orientation.z = quat[2]
        local_odom.pose.pose.orientation.w = quat[3]
        local_odom.pose.covariance[0]      = lon_stdev*lon_stdev
        local_odom.pose.covariance[7]      = lat_stdev*lat_stdev
        local_odom.pose.covariance[14]     = hgt_stdev*hgt_stdev
        local_odom.pose.covariance[21]     = rot_stdev*rot_stdev
        local_odom.pose.covariance[28]     = rot_stdev*rot_stdev
        local_odom.pose.covariance[35]     = rot_stdev*rot_stdev

        local_odom.twist.twist.linear.x    = hor_speed
        local_odom.twist.twist.linear.y    = 0.0
        local_odom.twist.twist.linear.z    = 0.0
        local_odom.twist.covariance[0]     = .125*.125
        local_odom.twist.covariance[7]     = .0025
        local_odom.twist.covariance[14]    = .0025

        local_odom_arr.append([ros_time, local_odom])

    return local_odom_arr


In [11]:
def process_bag(dir, bag, topics):

    # Read the bag file
    parser = BagFileParser(os.path.join(dir,bag))

    # Create Topic
    newid = len(parser.cursor.execute('SELECT * FROM topics').fetchall()) + 1
    topic_name = '/local_odometry'
    topic_type = 'nav_msgs/msg/Odometry'
    ser_form   = 'cdr'
    # oqp        = parser.cursor.execute('SELECT offered_qos_profiles FROM topics WHERE name = {}'.format('\'{}\''.format(topics['gps']))).fetchall()[0][0]
    oqp = ''
    
    # Insert Topic
    sql = 'INSERT INTO topics (id, name, type, serialization_format, offered_qos_profiles) VALUES (?, ?, ?, ?, ?)'
    input = (newid, topic_name, topic_type, ser_form, oqp)
    parser.cursor.execute(sql, input)

    # Generate Messages
    gps_messages = parser.get_messages(topics['gps'])
    vel_messages = parser.get_messages(topics['vel'])
    local_odom = gen_local_odom(gps_messages, vel_messages)

    # Insert Values
    num_msgs = parser.cursor.execute('SELECT COUNT(*) FROM messages').fetchall()[0][0]
    id_ind = num_msgs
    topic_id = newid

    for odom in tqdm(local_odom):
        id_ind += 1
        timestamp = odom[0]
        data = serialize_message(odom[1])
        sql = 'INSERT INTO messages (id, topic_id, timestamp, data) VALUES (?,?,?,?)'
        input = (id_ind, topic_id, timestamp, data)
        parser.cursor.execute(sql,input)

    parser.conn.commit()
    parser.__del__()


In [12]:
def reindex_bag(dir):
    # Reindex Bag
    serialization_format = 'cdr'
    storage_options = rosbag2_py.StorageOptions(uri=dir, storage_id='sqlite3')

    converter_options = rosbag2_py.ConverterOptions(
        input_serialization_format=serialization_format,
        output_serialization_format=serialization_format)

    reindexer = rosbag2_py.Reindexer()
    reindexer.reindex(storage_options)